# 制作你自己的 **AI** 动作识别系统

设备：香橙派 Zero 3 （同时执行太多任务可能会导致设备卡顿）

>本文档所有模块可以折叠，VS Code左下角点开**OUTLINE**查看目录可快速跳转

## 配置开发环境

In [1]:
# 导入所有库
import os
import cv2
import signal
import subprocess


In [2]:
# Get the current working directory
current_dir = os.getcwd()
print(f"Current directory: {current_dir}")
# Change the directory 切换执行路径
new_dir = "../inference_prediction"
os.chdir(new_dir)
# Verify the new directory 确认执行路径
changed_dir = os.getcwd()
print(f"Changed directory: {changed_dir}")

Current directory: c:\Users\PROMCRdog\Documents\Coding\Pi_Pose_Prediction\tutorials
Changed directory: c:\Users\PROMCRdog\Documents\Coding\Pi_Pose_Prediction\inference_prediction


In [ ]:
!sh setup.sh

## 初步测试

### 找到摄像头编号index

In [3]:
for camera_id in range(0, 20):
    cap = cv2.VideoCapture(camera_id)
    if cap is None or not cap.isOpened():
        # print(f"No camera found at index {camera_id}")
        continue
    else:
        ret, frame = cap.read()
        if ret:
            print(f"camera found at index {camera_id}")
        cap.release()

cv2.destroyWindow

camera found at index 0
camera found at index 1
camera found at index 2


<function destroyWindow>

### 测试

In [4]:
# Hardcoded variables 硬编变量
ESTIMATION_MODEL = 'movenet_lightning'  # 估计模型
TRACKER_TYPE = 'bounding_box'  # 跟踪器类型 ('keypoint' or 'bounding_box')
CLASSIFICATION_MODEL = 'classifier.tflite'  # Set to None if not using a classifier 如果不使用分类器，设置为None
LABEL_FILE = 'labels.txt'  # 标签文件
CAMERA_ID = 0  # 相机ID
WIDTH = 640  # 宽度
HEIGHT = 480  # 高度

In [5]:
# 执行文件
# Construct the command string
command = (
    f"python pose_estimation.py "
    f"--model {ESTIMATION_MODEL} "
    f"--tracker {TRACKER_TYPE} "
    f"--classifier {CLASSIFICATION_MODEL} "
    f"--label_file {LABEL_FILE} "
    f"--cameraId {CAMERA_ID} "
    # f"--frameWidth {WIDTH} "
    # f"--frameHeight {HEIGHT}"
)


print("Running command:", command)


try:
    # Run the command as a subprocess
    process = subprocess.Popen(command, shell=True)

    # Wait for the process to complete
    process.wait()
except KeyboardInterrupt:
    # Handle keyboard interrupt (Ctrl+C)
    print("\nProcess interrupted! Stopping...")
    process.send_signal(signal.SIGINT)
    process.terminate()
    process.wait()
    print("Process stopped.")

Running command: python pose_estimation.py --model movenet_lightning --tracker bounding_box --classifier classifier.tflite --label_file labels.txt --cameraId 0 


In [1]:
# Change back to initiall working directory 切换为ipynb原本所在路径
os.chdir(current_dir)

NameError: name 'os' is not defined